In [ ]:
# Cell 1: Install core libraries
%pip install --upgrade diffusers accelerate transformers safetensors huggingface_hub xformers torch torchvision

In [ ]:
# Cell 2: Mount Drive for persistent storage
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone the repository to your Google Drive
!git clone https://github.com/vanthomiy/a-me-generator.git
REPO_ROOT = "/content/a-me-generator"  # Replace with the actual name of your repo folder
!cp -r "$REPO_ROOT /

# it contains 
# |--content
#   |--class_images
#   |--generated_images
#   |--instance_images
# |--train_dreambooth_lora_sdxl.py

In [ ]:
# Cell 3: Login to Hugging Face Hub (to pull SDXL and push checkpoints)
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Cell 4: Define paths
INSTANCE_DIR = "/content/
CLASS_DIR    = "/content/drive/MyDrive/a-me-generator/class_images"     # optional generic faces
OUTPUT_DIR   = "/content/drive/MyDrive/a-me-generator/output_model"

# (Optional) Download or sync your images into INSTANCE_DIR before running.


In [ ]:
# Cell 5: Launch training
!accelerate launch train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --instance_data_dir=$INSTANCE_DIR \
  --class_data_dir=$CLASS_DIR \
  --instance_prompt="a photograph of my_face" \
  --class_prompt="a photograph of a person" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 \
  --max_train_steps=800 \
  --mixed_precision="fp16" \
  --enable_xformers_memory_efficient_attention \
  --output_dir=$OUTPUT_DIR


In [ ]:
# Cell 6: Load your fine‑tuned model
from diffusers import StableDiffusionXLImg2ImgPipeline, StableDiffusionXLPipeline
import torch

pipe = StableDiffusionXLPipeline.from_pretrained(
    OUTPUT_DIR,
    torch_dtype=torch.float16,
    use_safetensors=True,
).to("cuda")



In [ ]:
# Cell 7: Text-to-image generation
prompt = "A high‑resolution portrait of my_face standing in a sunlit garden, ultra realistic"
image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]
image.save("/content/drive/MyDrive/dreambooth/generated_portrait.png")
image.show()

In [ ]:
# Cell 8: Image-to-image generation
prompt = "A high‑resolution portrait of my_face standing in a sunlit garden, ultra realistic"
image = pipe(prompt, image=image, strength=0.75, num_inference_steps=50, guidance_scale=7.5).images[0]
image.save("/content/drive/MyDrive/dreambooth/generated_portrait_ii.png")
image.show()